In [1]:
import pickle
import os

os.chdir('..')
with open('window_data.pkl', 'rb') as handle:
    window_data = pickle.load(handle)
    
with open('window_data_cwi.pkl', 'rb') as handle:
    window_data_cwi = pickle.load(handle)

In [2]:
from scipy.stats import pearsonr
import pandas as pd
import csv
from scipy.stats import pearsonr
from scipy.stats import spearmanr

df_true = pd.read_csv('single_labels.tsv', sep='\t', quoting=csv.QUOTE_NONE)


In [3]:
df_test = pd.read_csv('data/raw/lcp_single_test.tsv', sep='\t', quoting=csv.QUOTE_NONE)

In [4]:
#Roberta
import pandas as pd

from transformers import RobertaTokenizer
from transformers import RobertaConfig, RobertaModel, AdamW
from transformers import get_linear_schedule_with_warmup
from transformers import RobertaForSequenceClassification
import random
import numpy as np
from transformers import get_linear_schedule_with_warmup
from sklearn.preprocessing import LabelEncoder
import torch 
from src.data import LexDataset
from torch.utils.data import DataLoader
from src.training import train, evaluate, forward_func_custom_bert, forward_func_baseline

epochs = 6
seed_val = 2
max_len = 22
stop_at = 7
cuda_card = 1

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
roberta_results_window = {}

for window_x in [1]:
    print('Window size of ', window_x)
    #data_texts, data_corpus, data_labels, sentence_raw, target_words, positions, _, pos_tags = window_data[window_x]['train']
    #news_texts, news_labels, news_target_words, news_positions, news_pos_tags = window_data_cwi[window_x]['news']
    #wikipedia_text, wikipedia_labels, wikipedia_target_words, wikipedia_positions, wikipedia_pos_tags = window_data_cwi[window_x]['wikipedia']
    #wikinew_texts, wikinew_labels, wikinew_target_words, wikinew_positions, wikinew_pos_tags = window_data_cwi[window_x]['wikinews']
    
    #pretraining_texts = np.concatenate((news_texts, wikipedia_text, wikinew_texts))
    #pretraining_labels = np.concatenate((news_labels, wikipedia_labels, wikinew_labels))
    #pretraining_target_words = np.concatenate((news_target_words, wikipedia_target_words, wikinew_target_words))
    #pretraining_positions = np.concatenate((news_positions, wikipedia_positions, wikinew_positions))
    #pretraining_pos_tags = np.concatenate((news_pos_tags, wikipedia_pos_tags, wikinew_pos_tags))
    
    train_texts, data_corpus, train_labels, sentence_train, train_target_words, train_positions, _, train_pos_tags = window_data[window_x]['train']
    multi_texts, multi_corpus, multi_labels, sentence_multi, multi_target_words, multi_positions, _, multi_pos_tags = window_data[window_x]['train_multi']
    data_texts = np.concatenate((train_texts, multi_texts))
    data_labels = np.concatenate((train_labels, multi_labels))
    sentence_raw = np.concatenate((sentence_train, sentence_multi))
    target_words = np.concatenate((train_target_words, multi_target_words))
    positions = np.concatenate((train_positions, multi_positions))
    pos_tags = np.concatenate((train_pos_tags, multi_pos_tags))
    
    trial_texts, _, trial_labels, sentences_trial, trial_target_words, trial_positions, _, trial_pos_tags = window_data[window_x]['val']
    test_texts, _, test_labels, sentences_test, test_target_words, test_positions, _, test_pos_tags= window_data[window_x]['test']
    
    le = LabelEncoder()
    le.fit(pos_tags)
    pos_tags_f = le.transform(pos_tags)
    trial_pos_tags_f = le.transform(trial_pos_tags)
    test_pos_tags_f = le.transform(test_pos_tags)
    #pretraining_pos_tags_f = le.transform(pretraining_pos_tags)

    pos_tags_f = np.reshape(pos_tags_f, (-1, 1))
    trial_pos_tags_f = np.reshape(trial_pos_tags_f, (-1, 1))
    test_pos_tags_f = np.reshape(test_pos_tags_f, (-1, 1))
    #pretraining_pos_tags_f = np.reshape(pretraining_pos_tags_f, (-1, 1))
    
    torch.cuda.manual_seed_all(seed_val)

    history_occur_text = []
    history_occur_target = []
    ix_fold = 0

    #train_index = list(range(len(pretraining_texts)))
    #random.seed(seed_val)
    #random.shuffle(train_index)
    #train_index = np.array(train_index)
    #pretrain_texts_lex, pretrain_labels_lex, pretrain_positions_lex, pretrain_target_words_lex, pretrain_pos_tags = pretraining_texts[train_index], pretraining_labels[train_index], pretraining_positions[train_index], pretraining_target_words[train_index], pretraining_pos_tags_f[train_index]

    train_index = list(range(len(data_texts)))
    random.seed(seed_val)
    random.shuffle(train_index)
    train_index = np.array(train_index)

    train_texts_lex, train_labels_lex, train_positions_lex, train_target_words_lex, train_pos_tags = data_texts[train_index], data_labels[train_index], positions[train_index], target_words[train_index], pos_tags_f[train_index]

    #pretrain_encodings_lex = roberta_tokenizer(list(pretrain_texts_lex), truncation=True, padding=True)
    #pretrain_dataset_lex = LexDataset(pretrain_encodings_lex, pretrain_labels_lex, pretrain_positions_lex)
    #pretrain_loader_lex = DataLoader(pretrain_dataset_lex, batch_size=8)
    
    train_encodings_lex = roberta_tokenizer(list(train_texts_lex), truncation=True, padding=True)
    train_dataset_lex = LexDataset(train_encodings_lex, train_labels_lex, train_positions_lex)
    train_loader_lex = DataLoader(train_dataset_lex, batch_size=32)
    
    train_encoding_aux = roberta_tokenizer(list(data_texts), truncation=True, padding=True)
    train_dataset_aux = LexDataset(train_encoding_aux, data_labels, positions)
    train_loader_aux = DataLoader(train_dataset_aux, batch_size=1)
    
    val_encodings_lex = roberta_tokenizer(list(trial_texts), truncation=True, padding=True)
    val_dataset_lex = LexDataset(val_encodings_lex, trial_labels, trial_positions)
    val_loader_lex = DataLoader(val_dataset_lex, batch_size=1)
    
    test_encodings_lex = roberta_tokenizer(list(test_texts), truncation=True, padding=True)
    test_dataset_lex = LexDataset(test_encodings_lex, test_labels, test_positions)
    test_loader_lex = DataLoader(test_dataset_lex, batch_size=1)
        
    all_text = ' '.join(data_texts)
    occurence_text = [all_text.count(w) for w in trial_target_words]
                
    device = torch.device(f"cuda:{cuda_card}")
    #device = torch.device(f"cpu")
        
    model = RobertaForSequenceClassification.from_pretrained(
        "roberta-base", 
        num_labels = 1, 
        output_attentions = False, 
        output_hidden_states = False, 
    )
    model.cuda(cuda_card)

    optimizer = AdamW(model.parameters(),
                      lr = 2e-5, 
                      eps = 1e-8
                      )   
    
    total_steps = len(train_loader_lex) * 3
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    random.seed(seed_val)
    torch.manual_seed(seed_val)

    #for epoch_i in range(0, 3):
    #    print()
    #    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    #    train(device, model, pretrain_loader_lex, forward_func_baseline, optimizer, scheduler, {})
    
    #total_steps = len(train_loader_lex) * epochs
    #scheduler = get_linear_schedule_with_warmup(optimizer, 
    #                                            num_warmup_steps = 0, # Default value in run_glue.py
    #                                            num_training_steps = total_steps)
    
    tracking_metrics_trn = []
    tracking_logits_trn = []
    
    tracking_metrics_val = []
    tracking_logits_val = []

    tracking_metrics_test = []
    tracking_logits_test = []
        
    early_stopping = 0
    best_metric = 10000
        
    for epoch_i in range(0, epochs):
        print()
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        train(device, model, train_loader_lex, forward_func_baseline, optimizer, scheduler, {})
            
        _, trn_logits, trn_metric = evaluate(device, model, train_loader_aux, forward_func_baseline, {})
        _, val_logits, val_metric = evaluate(device, model, val_loader_lex, forward_func_baseline, {})
        _, test_logits, test_metric = evaluate(device, model, test_loader_lex, forward_func_baseline, {})

        tracking_metrics_trn.append(trn_metric)
        tracking_logits_trn.append(trn_logits)
        
        tracking_metrics_val.append(val_metric)
        tracking_logits_val.append(val_logits)
        
        tracking_metrics_test.append(test_metric)
        tracking_logits_test.append(test_logits)
            
        if val_metric > best_metric:
            early_stopping += 1
        else:
            best_metric = val_metric
            early_stopping = 0
                
        if early_stopping == stop_at:
            break
       
    aux = []
    for i in range(epochs):
        aux.append(pearsonr(tracking_logits_val[i], trial_labels)[0])
        
    index_min = np.argmax(aux)

    ix_fold += 1

    print("")
    print("Training complete!")
    print(f"Best metric: {aux[index_min]}")
        
    del model
    torch.cuda.empty_cache()
        
    results_trn = pd.DataFrame()
    results_trn['samples'] = data_texts
    results_trn['preds'] = tracking_logits_trn[index_min]
    results_trn['true'] = data_labels
    
    results_val = pd.DataFrame()
    results_val['samples'] = trial_texts
    results_val['preds'] = tracking_logits_val[index_min]
    results_val['true'] = trial_labels
    
    results_test = pd.DataFrame()
    results_test['samples'] = test_texts
    results_test['preds'] = tracking_logits_test[index_min]
    results_test['true'] = test_labels

    roberta_results_window[window_x] = {'train': [], 'val': [], 'test': []}
    roberta_results_window[window_x]['train'] = results_trn    
    roberta_results_window[window_x]['val'] = results_val
    roberta_results_window[window_x]['test'] = results_test    

Using TensorFlow backend.


Window size of  1


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie


======== Epoch 1 / 6 ========
Training...

  Average training loss: 0.018231

Running Validation...
  Metric: 0.068799
  Validation Loss: 0.008063

Running Validation...
  Metric: 0.072310
  Validation Loss: 0.009061

Running Validation...
  Metric: 0.699603
  Validation Loss: 0.496928

======== Epoch 2 / 6 ========
Training...

  Average training loss: 0.011693

Running Validation...
  Metric: 0.065894
  Validation Loss: 0.007115

Running Validation...
  Metric: 0.070618
  Validation Loss: 0.008538

Running Validation...
  Metric: 0.680703
  Validation Loss: 0.474111

======== Epoch 3 / 6 ========
Training...

  Average training loss: 0.009858

Running Validation...
  Metric: 0.060878
  Validation Loss: 0.006117

Running Validation...
  Metric: 0.066740
  Validation Loss: 0.007621

Running Validation...
  Metric: 0.700404
  Validation Loss: 0.500882

======== Epoch 4 / 6 ========
Training...

  Average training loss: 0.009290

Running Validation...
  Metric: 0.060878
  Validation Los

In [5]:
df_test['preds'] = roberta_results_window[window_x]['test'].preds
df_cross = pd.merge(df_test, df_true, on=['id'])

df_cross[['id', 'preds']].to_csv('submit_1.csv', index=False, header=None)
pearsonr(df_cross.preds, df_cross.complexity)

(0.6903768286718371, 8.71519239679736e-131)

In [9]:
epochs = 5
seed_val = 2
max_len = 22
stop_at = 7
cuda_card = 1

roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base', do_lower_case=True)
roberta_pretrain_results_window = {}

for window_x in [1]:
    print('Window size of ', window_x)
    data_texts, data_corpus, data_labels, sentence_raw, target_words, positions, _, pos_tags = window_data[window_x]['train']
    news_texts, news_labels, news_target_words, news_positions, news_pos_tags = window_data_cwi[window_x]['news']
    wikipedia_text, wikipedia_labels, wikipedia_target_words, wikipedia_positions, wikipedia_pos_tags = window_data_cwi[window_x]['wikipedia']
    wikinew_texts, wikinew_labels, wikinew_target_words, wikinew_positions, wikinew_pos_tags = window_data_cwi[window_x]['wikinews']
    
    pretraining_texts = np.concatenate((news_texts, wikipedia_text, wikinew_texts))
    pretraining_labels = np.concatenate((news_labels, wikipedia_labels, wikinew_labels))
    pretraining_target_words = np.concatenate((news_target_words, wikipedia_target_words, wikinew_target_words))
    pretraining_positions = np.concatenate((news_positions, wikipedia_positions, wikinew_positions))
    pretraining_pos_tags = np.concatenate((news_pos_tags, wikipedia_pos_tags, wikinew_pos_tags))
    
    train_texts, data_corpus, train_labels, sentence_train, train_target_words, train_positions, _, train_pos_tags = window_data[window_x]['train']
    multi_texts, multi_corpus, multi_labels, sentence_multi, multi_target_words, multi_positions, _, multi_pos_tags = window_data[window_x]['train_multi']
    data_texts = np.concatenate((train_texts, multi_texts))
    data_labels = np.concatenate((train_labels, multi_labels))
    sentence_raw = np.concatenate((sentence_train, sentence_multi))
    target_words = np.concatenate((train_target_words, multi_target_words))
    positions = np.concatenate((train_positions, multi_positions))
    pos_tags = np.concatenate((train_pos_tags, multi_pos_tags))
    
    trial_texts, _, trial_labels, sentences_trial, trial_target_words, trial_positions, _, trial_pos_tags = window_data[window_x]['val']
    test_texts, _, test_labels, sentences_test, test_target_words, test_positions, _, test_pos_tags= window_data[window_x]['test']
    
    le = LabelEncoder()
    le.fit(pretraining_pos_tags)
    pos_tags_f = le.transform(pos_tags)
    trial_pos_tags_f = le.transform(trial_pos_tags)
    test_pos_tags_f = le.transform(test_pos_tags)
    pretraining_pos_tags_f = le.transform(pretraining_pos_tags)

    pos_tags_f = np.reshape(pos_tags_f, (-1, 1))
    trial_pos_tags_f = np.reshape(trial_pos_tags_f, (-1, 1))
    test_pos_tags_f = np.reshape(test_pos_tags_f, (-1, 1))
    pretraining_pos_tags_f = np.reshape(pretraining_pos_tags_f, (-1, 1))
    
    torch.cuda.manual_seed_all(seed_val)

    history_occur_text = []
    history_occur_target = []
    ix_fold = 0

    train_index = list(range(len(pretraining_texts)))
    random.seed(seed_val)
    random.shuffle(train_index)
    train_index = np.array(train_index)
    pretrain_texts_lex, pretrain_labels_lex, pretrain_positions_lex, pretrain_target_words_lex, pretrain_pos_tags = pretraining_texts[train_index], pretraining_labels[train_index], pretraining_positions[train_index], pretraining_target_words[train_index], pretraining_pos_tags_f[train_index]

    train_index = list(range(len(data_texts)))
    random.seed(seed_val)
    random.shuffle(train_index)
    train_index = np.array(train_index)

    train_texts_lex, train_labels_lex, train_positions_lex, train_target_words_lex, train_pos_tags = data_texts[train_index], data_labels[train_index], positions[train_index], target_words[train_index], pos_tags_f[train_index]
    
    pretrain_encodings_lex = roberta_tokenizer(list(pretrain_texts_lex), truncation=True, padding=True)
    
    train_encodings_lex = roberta_tokenizer(list(train_texts_lex), truncation=True, padding=True)
    train_dataset_lex = LexDataset(train_encodings_lex, train_labels_lex, train_positions_lex)
    train_loader_lex = DataLoader(train_dataset_lex, batch_size=32)
    
    train_encoding_aux = roberta_tokenizer(list(data_texts), truncation=True, padding=True)
    train_dataset_aux = LexDataset(train_encoding_aux, data_labels, positions)
    train_loader_aux = DataLoader(train_dataset_aux, batch_size=1)
    
    val_encodings_lex = roberta_tokenizer(list(trial_texts), truncation=True, padding=True)
    val_dataset_lex = LexDataset(val_encodings_lex, trial_labels, trial_positions)
    val_loader_lex = DataLoader(val_dataset_lex, batch_size=1)
    
    test_encodings_lex = roberta_tokenizer(list(test_texts), truncation=True, padding=True)
    test_dataset_lex = LexDataset(test_encodings_lex, test_labels, test_positions)
    test_loader_lex = DataLoader(test_dataset_lex, batch_size=1)
        
    all_text = ' '.join(data_texts)
    occurence_text = [all_text.count(w) for w in trial_target_words]
                
    device = torch.device(f"cuda:{cuda_card}")
    #device = torch.device(f"cpu")
        
    model = RobertaForSequenceClassification.from_pretrained(
        "roberta-base", 
        num_labels = 1, 
        output_attentions = False, 
        output_hidden_states = False, 
    )
    model.cuda(cuda_card)

    optimizer = AdamW(model.parameters(),
                      lr = 2e-5, 
                      eps = 1e-8
                      )   
    
    total_steps = len(train_loader_lex) * 3
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    random.seed(seed_val)
    torch.manual_seed(seed_val)

    for passing_id in range(4):
        roberta_pretrain_results_window[passing_id] = {}

        if passing_id:
            pretrain_labels_final = pretrain_labels_lex * 0.6 + np.array(tracking_pretrain_logits_test[index_min]) * 0.4
        else:
            pretrain_labels_final = pretrain_labels_lex
        
        pretrain_dataset_lex = LexDataset(pretrain_encodings_lex, pretrain_labels_final, pretrain_positions_lex)
        pretrain_loader_lex = DataLoader(pretrain_dataset_lex, batch_size=24)
        pretrain_pred_loader_lex = DataLoader(pretrain_dataset_lex, batch_size=1)
        
        for epoch_i in range(0, 2):
            print()
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, 2))
            train(device, model, pretrain_loader_lex, forward_func_baseline, optimizer, scheduler, {})

        total_steps = len(train_loader_lex) * epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                    num_warmup_steps = 0, # Default value in run_glue.py
                                                    num_training_steps = total_steps)

        tracking_metrics_trn = []
        tracking_logits_trn = []

        tracking_metrics_val = []
        tracking_logits_val = []

        tracking_metrics_test = []
        tracking_logits_test = []
        
        tracking_pretrain_metrics_test = []
        tracking_pretrain_logits_test = []

        early_stopping = 0
        best_metric = 10000

        for epoch_i in range(0, epochs):
            print()
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
            train(device, model, train_loader_lex, forward_func_baseline, optimizer, scheduler, {})

            #_, trn_logits, trn_metric = evaluate(device, model, train_loader_aux, forward_func_baseline, {})
            _, val_logits, val_metric = evaluate(device, model, val_loader_lex, forward_func_baseline, {})
            _, test_logits, test_metric = evaluate(device, model, test_loader_lex, forward_func_baseline, {})
            _, pretrain_logits, pretrain_metric = evaluate(device, model, pretrain_pred_loader_lex, forward_func_baseline, {})

            #tracking_metrics_trn.append(trn_metric)
            #tracking_logits_trn.append(trn_logits)

            tracking_metrics_val.append(val_metric)
            tracking_logits_val.append(val_logits)

            tracking_metrics_test.append(test_metric)
            tracking_logits_test.append(test_logits)
            
            tracking_pretrain_metrics_test.append(pretrain_metric)
            tracking_pretrain_logits_test.append(pretrain_logits)

            if val_metric > best_metric:
                early_stopping += 1
            else:
                best_metric = val_metric
                early_stopping = 0

            if early_stopping == stop_at:
                break

        aux = []
        for i in range(epochs):
            aux.append(pearsonr(tracking_logits_val[i], trial_labels)[0])

        index_min = np.argmax(aux)

        ix_fold += 1

        print("")
        print("Training complete!")
        print(f"Best metric: {aux[index_min]}")

        #torch.cuda.empty_cache()

        results_trn = pd.DataFrame()
        results_trn['samples'] = data_texts
        results_trn['preds'] = 1#tracking_logits_trn[index_min]
        results_trn['true'] = data_labels

        results_val = pd.DataFrame()
        results_val['samples'] = trial_texts
        results_val['preds'] = tracking_logits_val[index_min]
        results_val['true'] = trial_labels

        results_test = pd.DataFrame()
        results_test['samples'] = test_texts
        results_test['preds'] = tracking_logits_test[index_min]
        results_test['true'] = test_labels
        
        roberta_pretrain_results_window[passing_id][window_x] = {'train': [], 'val': [], 'test': []}
        roberta_pretrain_results_window[passing_id][window_x]['train'] = results_trn    
        roberta_pretrain_results_window[passing_id][window_x]['val'] = results_val
        roberta_pretrain_results_window[passing_id][window_x]['test'] = results_test    
        
        df_test['preds'] = roberta_pretrain_results_window[passing_id][window_x]['test'].preds
        df_cross = pd.merge(df_test, df_true, on=['id'])
        
        print('=========================')
        print('Preliminary results:', pearsonr(df_cross.preds, df_cross.complexity))
        print()

Window size of  1


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie


======== Epoch 1 / 2 ========
Training...

  Average training loss: 0.022626

======== Epoch 2 / 2 ========
Training...

  Average training loss: 0.016170

======== Epoch 1 / 5 ========
Training...

  Average training loss: 0.015601

Running Validation...
  Metric: 0.067958
  Validation Loss: 0.007753

Running Validation...
  Metric: 0.711809
  Validation Loss: 0.514285

Running Validation...
  Metric: 0.230103
  Validation Loss: 0.062778

======== Epoch 2 / 5 ========
Training...

  Average training loss: 0.010453

Running Validation...
  Metric: 0.067120
  Validation Loss: 0.007433

Running Validation...
  Metric: 0.693769
  Validation Loss: 0.493569

Running Validation...
  Metric: 0.238676
  Validation Loss: 0.069735

======== Epoch 3 / 5 ========
Training...

  Average training loss: 0.009295

Running Validation...
  Metric: 0.068336
  Validation Loss: 0.007820

Running Validation...
  Metric: 0.682838
  Validation Loss: 0.478538

Running Validation...
  Metric: 0.247697
  Valida

In [10]:
pearsonr(df_cross.preds, df_cross.complexity)

(0.7411180326057657, 1.6742373744273903e-160)

In [11]:
for passing_id in range(4):
    df_test['preds'] = roberta_pretrain_results_window[passing_id][window_x]['test'].preds
    df_cross = pd.merge(df_test, df_true, on=['id'])
        
    print('=========================')
    print('Preliminary results:', pearsonr(df_cross.preds, df_cross.complexity))

Preliminary results: (0.736343026795155, 2.0528055578848728e-157)
Preliminary results: (0.7446917077028818, 7.366476937367419e-163)
Preliminary results: (0.7400552278939332, 8.263070049470091e-160)
Preliminary results: (0.7411180326057657, 1.6742373744273903e-160)
